## data explopring and undersatanding 

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(df.shape)
print(df.head())
print('the colums are:')
print(df.columns.tolist())
print('the datatypes are:' , df.dtypes)
print('================================')
print ("the churn values are:")
print(df['Churn'].value_counts())
# converting the churnm values into 0 and 1 
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
print ('\n the churn values are converted successfully to binary ')
print('================================')
#customer tenbure distribution 
print('tenure statistics ')
print(df['tenure'].describe())
print('================================')
#monthly charges distribution 
print ('\n monthly charges distribution ')
print(df['MonthlyCharges'].describe())

print('================================')
#printing the null vbalues 
print('null values in the dataset are:')
print(df.isnull().sum())

#checking the empty values 
print('================================')
print (' the empty values are : ' , df.isin([' ', '']).sum())
#filling the empty values with 0
df['TotalCharges'] = df['TotalCharges'].replace(' ', '0').astype(float)
print('================================')
#checking the vaLUES AFTER FILLING 
null_count = df.isin([' ', '']).sum()
if null_count.sum() == 0:
    print('there is no empty values ')
else:
     print(df.isin([' ','']).sum())



(7043, 21)
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Co

## Making the model for the prediction 

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

#selecting feature for the model prediction 
features = ['tenure', 'MonthlyCharges', 'TotalCharges']
X = df[features]
y = df['Churn']
print  (f' the selected feature are: {features}')

#filling the null values with 0 
X = X.fillna(0)

#splitting the data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#training model 
model = LogisticRegression()
model.fit(X_train, y_train)

#making prediction 
y_pred = model.predict(X_test)
model.fit(X_train, y_train)


#ACCURACY SCORE 
accuracy = accuracy_score(y_test, y_pred)
print(f'the accuracy of the model is: {accuracy}')

#predicting nthe probality 
df['predicted_churn_prob'] = model.predict_proba(X)[:,1] #probality of leaving 
df['servival_prob'] =  1 - df['predicted_churn_prob']#probabilty of staying

#LTV = Monthly Charge × (1 / Churn Rate) × Discount Factor
#df['estimated_LTV'] = df['MonthlyCharges']* (1 / df['predicted_churn_prob'])* 0.8 (not giving proper values )
def calculate_realistic_ltv(monthly_charge , churn_prob , max_month = 60):
    expected_month = min(1 / churn_prob, max_month )
    discount_factor = 0.95
    ltv = monthly_charge * expected_month * discount_factor 
    return ltv
df['realistic_LTV'] = df.apply(
    lambda row : calculate_realistic_ltv(row['MonthlyCharges'], row['predicted_churn_prob']),
    axis = 1
)
print('Top 10 customer by estimated_LTV')
print (df[['customerID', "MonthlyCharges" , 'servival_prob','realistic_LTV' ]].sort_values('realistic_LTV', ascending = False ).head(10))
#checking churn proibabilty 
print ('churn probabilty rates:')
print (df['predicted_churn_prob'].describe())
#churn which is <5% 
low_risk = df[df['predicted_churn_prob']< 0.05]
print (f'customer eith <5% churn probability{len(low_risk)} ({len(low_risk)/ len(df):.1%})')
actual_churn_rate = df['Churn'].mean()
print(f"Actual churn rate in data: {actual_churn_rate:.1%}")

 the selected feature are: ['tenure', 'MonthlyCharges', 'TotalCharges']
the accuracy of the model is: 0.7977288857345636
Top 10 customer by estimated_LTV
      customerID  MonthlyCharges  servival_prob  realistic_LTV
6940  2314-TNDJQ           55.65       0.983029    3115.099731
6424  6034-ZRYCV           54.20       0.983611    3089.400000
786   4992-LTJNE           55.30       0.982981    3086.807026
2541  0739-UUAJR           53.80       0.983770    3066.600000
6661  3704-IEAXF           53.65       0.984178    3058.050000
206   6348-SNFUS           55.45       0.982483    3007.140264
3683  7688-AWMDX           54.50       0.982742    3000.005790
5014  3528-HFRIQ           52.30       0.983836    2981.100000
2899  6164-HAQTX           53.95       0.982778    2976.051457
3133  3181-VTHOE           51.05       0.983668    2909.850000
churn probabilty rates:
count    7043.000000
mean        0.265938
std         0.210754
min         0.004232
25%         0.085389
50%         0.212618
75%